<a href="https://colab.research.google.com/github/jamshid-ds/transcription_training/blob/main/stt_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Terminal cells

In [1]:
!mkdir audio #Audio folderining yaratib olinishi

In [ ]:
!pip install pydub

#Import cells

In [180]:
import pandas as pd
import numpy as np

from IPython.display import Audio

import librosa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, LinearRegression

import os

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#Kerakli kutubxonalar

#Data

In [138]:
transcription = pd.read_csv('/content/other.tsv', sep='\t', usecols=["sentence"])
transcription.head()

#Transkripsiya to'plangan DataFrame

,sentence
0,Tolibon jangsiz taslim bo‘lgan xavfsizlik xodi...
1,• Baxt hammaning hayotida bir marta bo‘ladi.
2,"• Sekin va’da qilib, tezda bajar."
3,"• Qo‘l ishlaydi, bosh topshiriq beradi."
4,Shifokor siri hisoblangan ma’lumotlarni qaysi ...


In [141]:
audio_file_path = '/content/audio/common_voice_uz_39618430.mp3.wav'

Audio(audio_file_path)

#Audio Datasetdan misol

##converting audios to wav format

In [142]:
transcription.shape

(122, 1)

In [143]:
!ls audio | wc -l

122


#Data Cleaning

In [121]:
import os

files = os.listdir("audio")

for file in files:
    if not file.endswith(".wav"):
        extension = file.split(".")[-1]

        !ffmpeg -i "audio/{file}" "audio/{file}.wav"

#Qulaylik uchun mp3 formatidan wav formatiga o'tildi

In [113]:
files = os.listdir("audio")

for file in files:
    if file.endswith(".mp3"):
        os.remove(os.path.join("audio", file))

#Yuqoradig cell yangi audiolarni yaratib beradi. Eski audiolarni o'chirib yuborish

##Removing any disallowed characters from the transcriptions

In [144]:
def remove_disallowed_characters(text):
  allowed_characters = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,!?' ")
  cleaned_text = ''.join([char for char in text if char in allowed_characters])
  return cleaned_text

transcription['sentence'] = transcription['sentence'].apply(remove_disallowed_characters)
transcription.head()

#Keraksi belgilarni o'chirib yuborish

,sentence
0,Tolibon jangsiz taslim bolgan xavfsizlik xodim...
1,Baxt hammaning hayotida bir marta boladi.
2,"Sekin vada qilib, tezda bajar."
3,"Qol ishlaydi, bosh topshiriq beradi."
4,Shifokor siri hisoblangan malumotlarni qaysi h...


##Converting all transcriptions to lowercase.

In [145]:
transcription["sentence"] = transcription["sentence"].apply(lambda x: x.lower())
transcription.head()

#Barcha so'zlarni kichik harflarga ko'chirish

,sentence
0,tolibon jangsiz taslim bolgan xavfsizlik xodim...
1,baxt hammaning hayotida bir marta boladi.
2,"sekin vada qilib, tezda bajar."
3,"qol ishlaydi, bosh topshiriq beradi."
4,shifokor siri hisoblangan malumotlarni qaysi h...


##Converting any numbers present in the transcriptions to strings.

In [146]:
def say(n) -> str:
    for i in range(4):
        if (n>=a[i]):
            return say(n//a[i])+b[i] + say(n%a[i])
    return c[n//10] + d[n%10]

a = (1000000000, 1000000, 1000,100)
b = ("milliard ", "million ", "ming ", "yuz ")
c = ("","o'n ","yigirma ", "o'ttiz ", "qirq ", "ellik ", "oltmish ", "yetmish ", "sakson ", "to'qson " )
d = ("", "bir ", "ikki ", "uch ", "to'rt ", "besh ", "olti ", "yetti ", "sakkiz ", "to'qqiz ")

#Sonlarni harfli sonlarga o'tkazish uchun funksiya qurildi

In [147]:
transcription['sentence'] = transcription['sentence'].apply(lambda x: say(int(x)) if x.isdigit() else x)
transcription.head()

#Yuqoridagi funksiyaning ishlatilishi

,sentence
0,tolibon jangsiz taslim bolgan xavfsizlik xodim...
1,baxt hammaning hayotida bir marta boladi.
2,"sekin vada qilib, tezda bajar."
3,"qol ishlaydi, bosh topshiriq beradi."
4,shifokor siri hisoblangan malumotlarni qaysi h...


##Converting audios to array format

In [148]:
import librosa

audio_arrays = []
for filename in os.listdir("audio"):
  if filename.endswith(".wav"):
    file_path = os.path.join("audio", filename)
    x, sr = librosa.load(file_path)
    audio_arrays.append(x)

transcription["audio_array"] = audio_arrays

#Transkripsiya va audioning array formati bitta dataframega joylandi

In [154]:
transcription.sample()

,sentence,audio_array
50,manavi yerimga urdi... dedim.,"[-5.162509e-13, -4.291834e-13, 1.1554929e-12, ..."
